In [5]:
!pip install onnxruntime

In [2]:
!pip install onnx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 28.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install onnxruntime



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install onnx onnxruntime numpy opencv-python scipy



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import onnx
import numpy as np
import cv2
from onnx import numpy_helper
from scipy.signal import correlate


In [9]:
import onnx
import numpy as np
import cv2
from onnx import numpy_helper
from scipy.signal import correlate
# -------- Load VGG16 ONNX and extract first conv layer weights --------
def extract_first_conv_weights(onnx_model_path):
    model = onnx.load(onnx_model_path)
    for initializer in model.graph.initializer:
        if "weight" in initializer.name and len(numpy_helper.to_array(initializer).shape) == 4:
            weights = numpy_helper.to_array(initializer)
            print(f"✅ Found 1st conv layer weights: {initializer.name}, shape: {weights.shape}")
            return weights
    raise ValueError("First conv layer weights not found")




# -------- Quantize weights using AbsMax method --------
def absmax_quantize(weights):
    absmax = np.max(np.abs(weights))
    scale = absmax / 127.0 if absmax != 0 else 1.0
    q_weights = np.clip(np.round(weights / scale), -127, 127).astype(np.int8)
    return q_weights, scale

# -------- Preprocess image like VGG16 (BGR & mean subtraction) --------
def preprocess_image(image_path):
    image = cv2.imread(image_path)  # BGR by default
    image = cv2.resize(image, (224, 224)).astype(np.float32)
    image[:, :, 0] -= 103.939
    image[:, :, 1] -= 116.779
    image[:, :, 2] -= 123.68
    image = np.transpose(image, (2, 0, 1))  # CHW
    return image

# -------- Manual convolution using INT8 weights --------
def conv2d_manual(input_image, q_weights, scale):
    out_channels, in_channels, kH, kW = q_weights.shape
    _, H, W = input_image.shape

    output = np.zeros((out_channels, H - kH + 1, W - kW + 1), dtype=np.float32)

    for out_ch in range(out_channels):
        for in_ch in range(in_channels):
            kernel = q_weights[out_ch, in_ch, :, :].astype(np.float32) * scale
            output[out_ch] += correlate(input_image[in_ch], kernel, mode='valid')

    return output

# -------- Main workflow --------
if __name__ == "__main__":
    onnx_model_path = "/kaggle/input/modeluplvgg/tensorflow2/default/1/vgg16-7.onnx"
    image_path = "/kaggle/input/reptiles/reptile.jpg"

    # Step 1: Get weights
    weights_fp32 = extract_first_conv_weights(onnx_model_path)

    # Step 2: Quantize weights to INT8
    q_weights, scale = absmax_quantize(weights_fp32)

    # Step 3: Load and preprocess image
    input_image = preprocess_image(image_path)

    # Step 4: Apply convolution manually
    conv_output = conv2d_manual(input_image, q_weights, scale)

    print(f"✅ Convolution Output Shape: {conv_output.shape}")
    print(f"🔍 Output (Channel 0, Center 5x5):\n{conv_output[0, :5, :5]}")


✅ Found 1st conv layer weights: vgg0_conv0_weight, shape: (64, 3, 3, 3)
✅ Convolution Output Shape: (64, 222, 222)
🔍 Output (Channel 0, Center 5x5):
[[-2.4891129 -2.4890823 -2.4891129 -2.4890823 -2.4891129]
 [-2.4890976 -2.4890976 -2.48909   -2.4890823 -2.4890823]
 [-2.48909   -2.4890823 -2.48909   -2.48909   -2.4890594]
 [-2.4891205 -2.4890976 -2.4891129 -2.4891129 -2.4890976]
 [-2.4890747 -2.4891052 -2.489067  -2.4890823 -2.4890976]]


In [12]:
def conv2d_fp32(input_image, weights_fp32):
    out_channels, in_channels, kH, kW = weights_fp32.shape
    _, H, W = input_image.shape

    output = np.zeros((out_channels, H - kH + 1, W - kW + 1), dtype=np.float32)

    for out_ch in range(out_channels):
        for in_ch in range(in_channels):
            kernel = weights_fp32[out_ch, in_ch, :, :]
            output[out_ch] += correlate(input_image[in_ch], kernel, mode='valid')

    return output

# Run float32 convolution
fp32_output = conv2d_fp32(input_image, weights_fp32)


In [15]:
!pip install scikit-image


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 23.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.5/226.5 kB 17.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [16]:
from math import log10
from skimage.metrics import structural_similarity as ssim

def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100.0
    max_val = np.max(img1)
    return round(20 * log10(max_val / np.sqrt(mse)), 2)

def calculate_ssim_map(img1, img2):
    img1 = img1.squeeze()
    img2 = img2.squeeze()

    # Mean over channels if needed
    if img1.ndim == 3:
        img1 = np.mean(img1, axis=0)
        img2 = np.mean(img2, axis=0)

    return round(ssim(img1, img2, data_range=img1.max() - img1.min()), 4)

In [18]:
psnr_score = calculate_psnr(fp32_output, conv_output)
ssim_score = calculate_ssim_map(fp32_output, conv_output)

print(f"📊 PSNR: {psnr_score} dB")
print(f"📊 SSIM: {ssim_score}")
print(f"🎯 Estimated Accuracy (via SSIM): {ssim_score * 100:.2f}%")


📊 PSNR: 50.74 dB
📊 SSIM: 0.9927
🎯 Estimated Accuracy (via SSIM): 99.27%
